# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [90]:
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
# First, put in your user name below
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!

MYTABLE = 'jdgendron' + '_data'


# Then, create a connection to the same database, using the 
# same user name and password in the `mysql-examples.ipynb` 
# file.  Use conn as the database connection variable.
import pymysql
from sqlalchemy import create_engine
import pandas as pd

host = 'slucor2020.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2020'
password = 'SLUcor2020'
database = 'hds5210'

conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)


In [91]:
assert(MYTABLE != 'paulboal_data')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [3]:
url = 'http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv'
data = pd.read_csv(url)
import getpass
myname = getpass.getuser().split('-')[1]
data.to_sql(myname + '_data', conn, index=False)

ValueError: Table 'jdgendron_data' already exists.

In [4]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (328,19))
assert(list(dxyz.columns) == ['Provider_Name', 'NPI', 'CCN',
       'Medicaid_EP_Hospital_Type', 'Street_Address', 'City', 'County',
       'State', 'Zip_Code', 'Payment_Year_Number', 'Program_Type',
       'Total_payments', 'Last_Program_Year', 'Last_Payment_Year',
       'Last_Payment_Criteria', 'Most_Recent_Disbursement_Amount', 'LONGITUDE',
       'LATITUDE', 'FID'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Zip_Code` field and `population`'s `zipcode` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [104]:
### SOLUTION GOES HERE
data_joined = pd.read_sql_query("""
  SELECT p.*, m.*
  FROM 
    population p JOIN
    jdgendron_data m ON p.zipcode = m.Zip_Code
""", conn)

data_joined = data_joined.sort_values('population', ascending = False).reset_index()
answer = data_joined['NPI'][0].astype(str)
answer = [answer]

In [105]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

In [103]:
df_zip = pd.read_sql_query("SELECT * FROM jdgendron_data", conn)
df_zip = df_zip.groupby('Zip_Code').sum().reset_index()
df_ht = pd.read_sql_query("SELECT * FROM jdgendron_data", conn)
df_ht = df_ht.groupby('Medicaid_EP_Hospital_Type').sum().reset_index()
df_ht = df_ht[['Medicaid_EP_Hospital_Type','Total_payments']]
df_pop = pd.read_sql_query("SELECT * FROM population", conn)
df_pop['zipcode'] = df_pop['zipcode'].astype(int)
df_join = df_pop.merge(df_zip, left_on='zipcode', right_on='Zip_Code')
df_join['Avg_Pay_per_Capita'] = df_join['Total_payments'] / df_join['population']
df_join
## I couldn't figure out this one. I got the avg_pay_per_capita but I wasn't sure exactly how to get the hospital type data to merge with it based on the columns
## available. Sorry I couldn't get this one.

,Zip_Code,NPI,CCN,Payment_Year_Number,Total_payments,Last_Program_Year,Last_Payment_Year,Most_Recent_Disbursement_Amount,LONGITUDE,LATITUDE,FID
0,90004,1639166200,50111,2,1292117,2013,2013,484544,-118.283666,34.075313,213
1,90012,1881656460,50018,4,3840496,2015,2016,384050,-118.239093,34.064563,71
2,90015,1114081056,50149,1,5580824,2015,2015,5580824,-118.265408,34.037180,252
3,90017,1508859323,50471,4,4195996,2014,2015,419600,-118.265299,34.053179,179
4,90018,1639195175,50644,1,1553491,2012,2013,1553491,-118.308991,34.035978,214
...,...,...,...,...,...,...,...,...,...,...,...
283,96104,1801973052,51308,2,184615,2013,2014,69231,-120.171904,41.533282,51
284,96122,1326044140,51300,4,515430,2015,2016,51543,-120.470526,39.805198,322
285,96130,1538197231,51320,4,1323384,2014,2015,132338,-120.649774,40.438386,197
286,96150,1003867565,50352,2,350678,2016,2018,131504,-119.998043,38.912336,220


In [ ]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 17974.363)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 111.559)
